Our data is from 2 source zip files named trec07 and 1998. They are unstructured text files, that contain email headers, subject and body. In this part, we parse through the text files, convert into a structured pandas dataset, and save it as "structured.xlsx"

### If on Google Colab

Execute the following cells only if running on Google Colab, to install the needed packages and download the datasets from Google drive.

In [1]:
!pip install PyDrive
!pip install XlsxWriter

    100% |████████████████████████████████| 993kB 9.1MB/s 
  Running setup.py bdist_wheel for PyDrive ... - \ done
  Stored in directory: /content/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive
    100% |████████████████████████████████| 143kB 5.5MB/s 


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authenticate PyDrive API to access Google drive

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Now, we download and untar the two datasets, 1998 and trec07p, that we are going to work with.

In [0]:
download = drive.CreateFile({'id': '1QtoxpJmd1lys7c7LaYXiOjbzMdMOpeVX'})
download.GetContentFile('1998.tar')

In [0]:
download2 = drive.CreateFile({'id': '1xaJL1eoccrCyS45xgF23dVY_KCER-oAD'})
download2.GetContentFile('trec07p.tar')

In [0]:
!tar xf 1998.tar
!tar xf trec07p.tar

### If not on Google Colab

If running locally, make sure to have the 1998 dataset and trec07p dataset in the same directory as this notebook.

In [1]:
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

NotHeaders contains a list of words that do not qualify as an email header

In [2]:
notHeaders = ['A',
 'A.normal2{color',
 'A.normal{color',
 'A.over{color',
 'ATT',
 'Action',
 'Added',
 'Address',
 'An',
 'Asunto',
 'Author',
 'Auto-Submitted',
 'BATCH',
 'Betreff',
 'Beurteilung',
 'Bookmark',
 'Brma',
 'BrmaSmtpAuthUser',
 'C',
 'Call',
 'Cancel-Lock',
 'Carlson',
 'Caveats',
 'Changeset',
 'Cia1iis',
 'Cited',
 'Classification',
 'Comment',
 'Company',
 'Complaints',
 'Congratulations',
 'Contact',
 'Copyright',
 'Corp.<http',
 'Corporation<http',
 'Credibility',
 'Date-warning',
 'Datum',
 'De',
 'Details',
 'De\xa0',
 'Diagnostic-Code',
 'Dinah',
 'Envoyé\xa0',
 'FATAL',
 'FDA',
 'FONT-SIZE',
 'Featuring',
 'Fixes',
 'Gesendet',
 'Hash',
 'Hinweis',
 'ISIN',
 'Importance',
 'Inc.<http',
 'Index',
 'Jabber-ID',
 'Jim',
 'John',
 'Kopia',
 'LINE-HEIGHT',
 'List-Help',
 'Log',
 'London',
 'Lookup',
 'MOTD',
 'Mail-Followup-To',
 'Managed-by',
 'Market',
 'Metze',
 'Modified',
 'NOTE',
 'Name',
 'Napster<http',
 'Newshawk',
 'Notice',
 'Number',
 'OTC',
 'Objet\xa0',
 'Old-Return-Path',
 'OpenPGP',
 'Organisation',
 'Organization',
 'PADDING-BOTTOM',
 'PHONE',
 'Package',
 'Page',
 'Para',
 'Phone',
 'Posted',
 'Precedence',
 'President',
 'Priority',
 'Products',
 'Pubdate',
 'Publicitate',
 'REF',
 'RT-Ticket',
 'Rangel',
 'Received-SPF',
 'References',
 'Reminder',
 'Removed',
 'Reporting-MTA',
 'S.umbol',
 'Sent',
 'Severity',
 'Site',
 'Skickat',
 'Source',
 'Spoken',
 'Sym8oL',
 'Symbol',
 'Sys.putenv("http_proxy"="http',
 'TELEPHONE',
 'Talk',
 'Tel',
 'Teste',
 'Ticker',
 'Till',
 'Timing',
 'Tname',
 'Try',
 'Type',
 'UEI',
 'URL',
 'Visit',
 'Von',
 'WASHINGTON',
 'WKN',
 'Webpage',
 'Website',
'A.RVTS2',
 'ASTIG',
 'AXIS',
 'Ambieen',
 'Anova',
 'Below',
 'CARGO',
 'CNN',
 'CNNMoney',
 'City',
 'Clinton',
 'Collins',
 'Given',
 'HEADLINES',
 'IRAN',
 'Iterations',
 'Juego!<o',
 'Kernel',
 'L<Parrot',
 'Parrot',
 'Price']

Since many email body fields contain HTML markup too, we define a method that filters out visible text from HTML content.

In [3]:
def parseTextFromHTML(body):
    '''
    eliminate HTML markup tags and return only content
    '''
    body = body.lower()
    htmlstartindex = body.find('<html>')
    htmlendindex = body.find('</html>')
    
    #if(htmlstartindex == -1 or htmlendindex == -1):
        #return body
    
    htmlcontent = body#[htmlstartindex:htmlendindex+7]
    soup = BeautifulSoup(htmlcontent, 'html.parser')
    [s.extract() for s in soup(['style', 'script'])]
    return soup.getText().strip()
    #return body[:htmlstartindex] + soup.getText().strip() + body[htmlendindex+7:]

In [4]:
"""# parse the email file(spam or ham) to create a pandas dataframe
def getDFFromEmail(path, spam):
    l = []
    precurrkv = []
    sufcurrkv = []
    count = 0
    body = False
    try:
        stream = open(path, errors='strict', encoding='UTF-8')
        stream.readlines()
        stream.seek(0)
        print("Using UTF-8 encoding")
    except (Exception):
        print("Using ANSI encoding")
        stream = open(path, errors='strict', encoding='ANSI')
        stream.readlines()
        stream.seek(0)
        
    for line in stream:
        if(line.startswith('<DOCTYPE')):
            continue
            
        colonIndex = line.find(":")
        
        if(count > 15 and not body):
            body = True
            l.append(precurrkv)
            
        if(line[0].isupper() and line.find(' ',0,colonIndex)<0 and colonIndex >= 0 and colonIndex < 30 and not body 
           and line[:colonIndex] not in notHeaders):
            count = 0
            if(len(sufcurrkv) > 0 and len(precurrkv) > 0):
                precurrkv[1] = "".join((precurrkv[1], "".join(sufcurrkv)))
                l.append(precurrkv)
            elif(len(precurrkv) > 0):
                l.append(precurrkv)
            precurrkv = line.split(sep=":", maxsplit=1)
            sufcurrkv.clear()

        elif(str(line[0:2]).isspace() and not body):
            count += 1
            sufcurrkv.append(line)
        elif(str(line).startswith('>') and not body):
            body = True
            sufcurrkv = [''.join((':'.join(precurrkv), "".join(sufcurrkv)))]
        else:
            #count += 1
            sufcurrkv.append(line)

    l.append(["Body", parseTextFromHTML("".join(sufcurrkv))])
    l.append(["Spam", 'Spam' if spam else 'Ham'])
    l.append(["Tname", path])

    d = pd.DataFrame(np.array(l)).drop_duplicates(subset=0)
    return d    """
""""""

''

We define a method that parses an email text file represented by the path parameter and returns a pandas dataframe.

In [5]:
# parse the email file(spam or ham) to create a pandas dataframe
from collections import deque
from codecs import open
def getDFFromEmail(path, spam):
    '''
    path is path to file
    spam is a boolean indicating spam or ham (not spam)
    returns a dataframe containing email fields and their values for given path's file
    '''
    l = dict() #used for parsing text into dict of keys and values
    precurrkv = []  #list of keys, for eg in Sender : Vighnesh, here sender is key and Vighnesh is value
    sufcurrkv = deque() #list of values
    count = 0 #character index of read line
    body = False #indicates that body has started
  
#figure out the correct encoding using try except
    try:
        stream = open(path, errors='strict')
        stream.readlines()
        stream.seek(0)
        #print("Using UTF-8 encoding")
    except (Exception):
        try:
            #print("Using ISO-8859-1 encoding")
            stream = open(path, errors='strict', encoding='iso-8859-1')
            stream.readlines()
            stream.seek(0)
        except (Exception):
            return ""
        

    for line in stream:
        #skip doctype statements
        if(line.startswith('<DOCTYPE')):
            continue
            
        colonIndex = line.find(":")
        
        #line is part of body
        if(count > 15 and not body):
            body = True

        #check if new email header has been read
        # first letter isupper for header,no spaces in header,colonIndex is within bounds and header is not in notHeaders
        if(line[0].isupper() and line.find(' ',0,colonIndex)<0 and colonIndex >= 0 and colonIndex < 30 and not body 
           and line[:colonIndex] not in notHeaders):
            count = 0 #reset index to 0
            #append value to key in dictionary
            if(sufcurrkv and len(precurrkv) > 0):
                fieldname = sufcurrkv.popleft()
                l[fieldname] = [''.join(sufcurrkv)]
            
            precurrkv = line.split(sep=":", maxsplit=1)
            sufcurrkv.clear()
            sufcurrkv.extend(precurrkv)

        #read line is part of previous header
        elif(str(line[0:2]).isspace() and not body):
            count += 1
            sufcurrkv.append(line)
            
        # > indicates previous email characters, thus body has started
        elif(str(line).startswith('>') and not body):
            body = True
            sufcurrkv.append(line)
            
        # body has started
        else:
            #count += 1
            sufcurrkv.append(line)

    #create keys and values for body, spam and filename
    fieldname = sufcurrkv.popleft()
    l[fieldname] = sufcurrkv.popleft()    
    l["Body"] = [parseTextFromHTML("".join(sufcurrkv))]
    l["Spam"] = ['Spam'] if spam else ['Ham']
    l["Tname"] = [path]

    d = pd.DataFrame(l)
    return d.T.reset_index()    

We define a generator for the 1998 dataset that iterates over all email text files in a directory and yields resulting dataframes.

In [6]:
# iterate over all files in the directory to create the dataframes
def getAllDFFromDirectory(directorypath):
    '''
    directorypath is path to 1998 dataset directory
    '''
    for filename in os.listdir(directorypath):
        #print("Getting DF for "+"".join((directorypath, filename)))
        if(filename.startswith("spm")):
            yield getDFFromEmail("".join((directorypath, filename)), True)
        elif(filename.count("msg") > 0):
            yield getDFFromEmail("".join((directorypath, filename)), False)
        else:
            yield getDFFromEmail("".join((directorypath, filename)), True)

In [7]:
from functools import reduce

# create dataframes from all directories from that contain spam or ham email text files

In [8]:
directorypath = "1998/1998/03/"
dfs = [i for i in getAllDFFromDirectory(directorypath) if(isinstance(i,pd.DataFrame))]

We define a generator for the trec07p dataset that iterates over all email text files in a directory and yields dataframes.

In [9]:
# iterate over all files in the directory to create the dataframes
def getAllDFFromDirectory2(directorypath, target):
    '''
    passes filename and target(spam or ham) to getDFFromEmail function explained above
    '''
    for i,filename in enumerate(os.listdir(directorypath)[:5000]):
        #print("Getting DF for "+"".join((directorypath, filename)))
        
        if(target[i] == 'spam'):
            yield getDFFromEmail("".join((directorypath, filename)), True)
        else:
            yield getDFFromEmail("".join((directorypath, filename)), False)

In [10]:
def getTrec07Target():
    '''
    index file has target(spam or ham) and path to data
    '''
    for line in open("trec07p/trec07p/full/index"):
        yield line.split(" ")[0]

target = [i for i in getTrec07Target()]
print(len(target))

directorypath = "trec07p/trec07p/data/"
dfs2 = [i for i in getAllDFFromDirectory2(directorypath, target) if(isinstance(i,pd.DataFrame))]

75419


In [11]:
# Test for number of columns in dataframes
#dfs2.map(lambda df: df.shape[0])
#[i for i in zip(map(lambda df: df.shape[0],dfs2), range(len(dfs2))) if(i[0]>40)]


Use functools.reduce to merge all resulting dataframes into a single dataframe representing all emails.

In [12]:
df_final = reduce(lambda left,right: pd.merge(left,right,how='outer',on='index'), dfs+dfs2)

In [13]:
df_final = df_final.set_index('index').T

In [14]:
df_final['index'] = [i for i in range(df_final.shape[0])]
df_final.set_index('index', inplace=True)

In [15]:
df_final.head()

index,Return-Path,Delivered-To,Received,From,To,Message-Id,Reply-To,Subject,Mime-Version,Content-Type,...,X-PMX-Version-Mac,X-PerlMx-Spam,X-Sagator-Scanner,X-Sagator-ID,Nos,X-AuditID,Content-Language,SINGAPORE,X-imss-approveListMatch,X-twelveapples.com-MsgID
index,,,,,,,,,,,,,,,,,,,,,
0,<aj881c@ix.netcom.com>\n,bait@mikhail.qcc.sk.ca\n,from local.nethost.org(really [24553/24554]) ...,aj881c <aj881c@ix.netcom.com>\n,<bagpipes@acadia.net>\n,<19943672.886214@relay.comanche.denmark.eu> M...,aj881c@ix.netcom.com\nAuthenticated sender is...,2-1\n,1.0\n,"text/plain; charset=""us-ascii""\n",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,<iwbp@mailcity.com>\n,bait@mikhail.qcc.sk.ca\n,from mail.hic.net (1Cust113.tnt8.lax3.da.uu.n...,iwbp@mailcity.com\n,members@your.net\n,NaN,NaN,"Exclusive Internet Business, 1st Time Offered...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,<am74rt@worldnet.att.net>\n,bait@mikhail.qcc.sk.ca\n,from local.nethost.org(really [24553/24554]) ...,am74rt <am74rt@worldnet.att.net>\n,<badams@eastky.com>\n,<19943672.886214@relay.comanche.denmark.eu> T...,am74rt@worldnet.att.net\n\nAuthenticated send...,2-17\n,1.0\n,"text/plain; charset=""us-ascii""\n",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,<subwiz1@friendlyserver.com>\n,bait@mikhail.qcc.sk.ca\n,from subwiz1@friendlyserver.com œby net1.aoci...,"""D.Reynolds"" <subwiz1@friendlyserver.com>\n",NaN,<199802161222.EAA24869@net1.aoci.com>\n,subwiz1@friendlyserver.com\n,ADV: FREE DOWNLOAD:Register your web site to ...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,<carlover@goplay.com>\n,bait@mikhail.qcc.sk.ca\n,from mail.anet-chi.com (1Cust245.tnt13.atl2.d...,carlover@goplay.com\n,carlovers@america.com\n,NaN,NaN,AUTOMOBILE OPPORTUNITY\n,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Notice that some of the columns are duplicates, for eg. 'Message-Id', 'Message-ID' & 'Message-id', so we define a method to combine these duplicate columns.

In [16]:
def combineDuplicateColumns(df_final, colNames):
    '''
    combines all colNames to 1 and drops duplicate columns
    '''
    originalCol = ''
    for i,duplicateCol in enumerate(colNames):
        if(i == 0):
            originalCol = duplicateCol
            continue
        
        df_final.loc[df_final[originalCol].isna(), originalCol] = df_final.loc[df_final[originalCol].isna()][duplicateCol]
    
    
    df_final.drop(labels=colNames[1:], axis=1, inplace=True)

In [17]:
# Combine duplicate columns
combineDuplicateColumns(df_final, ['Message-Id', 'Message-ID', 'Message-id'])
combineDuplicateColumns(df_final, ['Reply-To', 'Reply-to'])
combineDuplicateColumns(df_final, ['Mime-Version', 'MIME-version', 'MIME-Version'])
combineDuplicateColumns(df_final, ['Content-Type', 'Content-type'])
combineDuplicateColumns(df_final, ['Content-Transfer-Encoding', 'Content-transfer-encoding'])
combineDuplicateColumns(df_final, ['Error-To', 'Errors-To', 'Errors-to'])
combineDuplicateColumns(df_final, ['Content-Length','Content-length'])

Now that all duplicate columns are eliminated, lets display all columns of our final dataframe.

In [18]:
# Check for more duplicate columns
np.sort(df_final.columns.values)

array(['Account', 'Addressee', 'Amount', 'Approved-By', 'Architecture',
       'Army', 'Arrival-Date', 'Association', 'Attn',
       'Authentication-Results', 'Availability', 'Bcc', 'Body', 'Boxer',
       'BroadcastJobID', 'BroadcastRecipientID', 'CC', 'COMM', 'COMMENTS',
       'Cc', 'Cialis', 'Coefficients', 'Comments', 'Conf', 'Content-Base',
       'Content-Class', 'Content-Description', 'Content-Disposition',
       'Content-ID', 'Content-Language', 'Content-Length',
       'Content-Location', 'Content-MD5', 'Content-Transfer-Encoding',
       'Content-Type', 'Content-class', 'Content-description',
       'Content-disposition', 'Content-language', 'Copied',
       'DKIM-Signature', 'DSN', 'Date', 'Delivered-To', 'Design',
       'Development', 'Disposition-Notification-To', 'Dobbs',
       'DomainKey-Signature', 'DomainKey-Status', 'E-Mail', 'E-mail',
       'EMAIL', 'ERROR', 'EXTRAS.</span></font><o', 'Email', 'Error',
       'Error-To', 'FACT', 'FROM', 'Fax', 'Final-Recipient',

Let's keep the columns that we need and eliminate the rest.

In [19]:
#From, To, Body, Subject, Message-Id, X-UIDL, Sender, Spam
c1 = df_final.columns
c2 = pd.Index(['Spam', 'Body', 'Subject', 'From', 'To', 'Message-Id', 'X-UIDL', 'Sender'])
c1 = c1.difference(c2)

df_new = df_final.drop(labels=c1.format(), axis=1, inplace=False)
df_new.columns

Index(['From', 'To', 'Message-Id', 'Subject', 'Body', 'Spam', 'X-UIDL',
       'Sender'],
      dtype='object', name='index')

Here is our final dataframe. We now have data in a structured format. Saving it as "structured.xlsx". 

In [20]:
df_new.head()

index,From,To,Message-Id,Subject,Body,Spam,X-UIDL,Sender
index,,,,,,,,
0,aj881c <aj881c@ix.netcom.com>\n,<bagpipes@acadia.net>\n,<19943672.886214@relay.comanche.denmark.eu> M...,2-1\n,email marketing works!!\n\nbull's eye gold is ...,Spam,NaN,NaN
1,iwbp@mailcity.com\n,members@your.net\n,<>\n,"Exclusive Internet Business, 1st Time Offered...",>>>this is the most exciting breakthrough ever...,Spam,NaN,NaN
2,am74rt <am74rt@worldnet.att.net>\n,<badams@eastky.com>\n,<19943672.886214@relay.comanche.denmark.eu> T...,2-17\n,email marketing works!!\n\nbull's eye gold is ...,Spam,NaN,NaN
3,"""D.Reynolds"" <subwiz1@friendlyserver.com>\n",NaN,<199802161222.EAA24869@net1.aoci.com>\n,ADV: FREE DOWNLOAD:Register your web site to ...,free download.register your web site to over 7...,Spam,NaN,NaN
4,carlover@goplay.com\n,carlovers@america.com\n,<>\n,AUTOMOBILE OPPORTUNITY\n,do you love cars?\n\nwant your own business?\n...,Spam,NaN,NaN


In [21]:
#Save to excel 
excelwriter = pd.ExcelWriter('structured.xlsx', engine='xlsxwriter')

df_new.to_excel(excelwriter, index=False)

excelwriter.save()

C:\Users\kunal\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:830: UserWarning: Ignoring URL 'http://fromgeneva.blogspot.com/2007/04/pre-development-agenda-meeting-
in.html

pre-development%20agenda%20meeting%20in%20singapore

fromgeneva
thiru%20balasubramaniam
16%20april%202007

the%20government%20of%20singapore%20will%20host%20a%20pre-development%20agenda%20meeting
from%2030%20may%20to%201%20june%202007.%20the%20meeting%20is%20being%20convened%20by%20the
intellectual%20property%20office%20of%20singapore%20(ipos)%20with%20invitations%20being
signed%20by%20ms.%20liew%20woon%20yin,%20director-general%20(ipos).

according%20to%20the%20invitation%20which%20was%20sent%20out%20on%2029%20march%202007,%20the
singaporean%20government%20is%20encouraged%20by%20positive%20outcomes%20achieved%20in
the%203rd%20session%20of%20the%20provisional%20committee%20on%20proposals%20related%20to%20a
wipo%20development%20agenda%20(pcda).%20in%20order%20to%20%22ensure%20that%20the%20pcda
succeeds%20in%20i

C:\Users\kunal\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:830: UserWarning: Ignoring URL 'http://www.kaisernetwork.org/daily_reports/rep_index.cfm?dr_id=44254

thailand%20will%20maintain%20compulsory%20licenses%20for%20kaletra,%20efavirenz,
despite%20companies'%20drug%20price%20reductions,%20health%20minister%20says

kaiser%20news%20network
apr%2016,%202007

thailand%20will%20maintain%20compulsory%20licenses%20for%20abbott%20laboratories'
antiretroviral%20drug%20kaletra%20and%20merck's%20antiretroviral%20efavirenz,
despite%20both%20companies'%20decision%20to%20reduce%20the%20cost%20of%20both%20drugs,%20thai
health%20minister%20mongkol%20na%20songkhla%20said%20on%20thursday,%20reuters%20health
reports%20(wong-anan,%20reuters%20health,%204/12).%20mongkol%20in%20january%20signed%20the
compulsory%20license,%20which%20allows%20thailand%20to%20produce%20a%20lower-cost
version%20of%20kaletra,%20into%20law.%20abbott%20in%20march%20announced%20that%20it%20had
withdrawn%20applications%2

Download to local if using Google Colab

In [ ]:
from google.colab import files
files.download('structured.xlsx')